In [1]:
!head SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

# Building SciKit-Learn Compatible Transfomers

If we use the methods `.fit(self, X, y=None)` and `.transform(self, X)`, we can build transformers that will easily create feature vectors in SciKit-Learn that we can just pop into a Pipeline, and do things more automatically.

We'll look at SMS spam data to demonestrate how to do this.

In [3]:
import csv
import re
import numpy as np
import random

from sklearn.pipeline import make_pipeline, make_union

In [4]:

sms_data = []
sms_results = []

with open('SMSSpamCollection') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        sms_data.append(row[1])
        sms_results.append(row[0])

In [5]:
sms_data[:5]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though"]

In [6]:
sms_results[:5]

['ham', 'ham', 'spam', 'ham', 'ham']

Let's start by writing a very dumb featurizer/vectorizer. It will just return a vector [1] for every result.

It's a good demonstration of the minimum possible featurizer

In [10]:
from sklearn.base import TransformerMixin

class DumbFeaturizer(TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [[1] for _ in X]

In [11]:
from sklearn.tree import DecisionTreeClassifier

pipe = make_pipeline(DumbFeaturizer(), DecisionTreeClassifier())
pipe.fit(sms_data, sms_results)
pipe.score(sms_data, sms_results)

0.86593682699210339

In [14]:
sms_results.count('ham') / len(sms_results)

0.8659368269921034

Our baseline, making a guess of the most common result, gives us ham every time, which is right 87% of the time.

We need to make sure our model is better than our baseline.

Let's build a better featurizer to try and get better results

In [33]:
def longest_run_of_capital_letters(text):
    '''Find the longest run of capital letters and return their length'''
    text = re.sub(r'\W', '', text)
    result = re.findall(r'[A-Z]+', text)
    if result:
        return len(sorted(result, key=len, reverse=True)[0])
    else:
        return 0

In [25]:
text = 'My VERY excellent MOTHEr juST servED us NINE PIZZAS'
text = re.sub(r'\W', '', text)
result = re.findall(r'[A-Z]+', text)

In [26]:
result

['M', 'VERY', 'MOTHE', 'ST', 'ED', 'NINEPIZZAS']

In [27]:
len(sorted(result, key=len, reverse=True)[0])

10

In [29]:
class SingleFunctionFeaturizer(TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        '''All SciKit-learn compatible transformers and classifiers have the same
        interface. `fit` should always return the same object (self)'''
        return self
    
    def transform(self, X):
        '''Given a list of original data, return a list of feature vectors'''
        return [[longest_run_of_capital_letters(x)] for x in X]

In [30]:
sms_featurizer = SingleFunctionFeaturizer()
sms_featurizer.transform(sms_data[:10])



[[1], [1], [3], [1], [1], [1], [1], [1], [7], [4]]

In [31]:
sms_data[:10]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though",
 "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv",
 'Even my brother is not like to speak with me. They treat me like aids patent.',
 "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune",
 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.',
 'Had you

In [34]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sms_data, sms_results)

pipe = make_pipeline(sms_featurizer, DecisionTreeClassifier())
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.91529073941134242

In [35]:
class FunctionFeaturizer(TransformerMixin):
    def __init__(self, *featurizers):
        self.featurizers = featurizers
        
    def fit(self, X, y=None):
        '''All SciKit-learn compatible transformers and classifiers have the same
        interface. `fit` should always return the same object (self)'''
        return self
    
    def transform(self, X):
        '''Given a list of original data, return a list of feature vectors'''
        feature_vectors = []
        for x in X:
            feature_vector = [f(x) for f in self.featurizers]
            feature_vectors.append(feature_vector)
        
        return np.array(feature_vectors)


In [39]:
def percentage_of_punctuation(text):
    total_length = len(text)
    text = re.sub(r'[\w\s]', '', text)
    punc_length = len(text)
    
    return punc_length / total_length

f = FunctionFeaturizer(len,
                       longest_run_of_capital_letters,
                       percentage_of_punctuation)
f.transform(sms_data[:10])

array([[  1.11000000e+02,   1.00000000e+00,   8.10810811e-02],
       [  2.90000000e+01,   1.00000000e+00,   2.06896552e-01],
       [  1.55000000e+02,   3.00000000e+00,   3.87096774e-02],
       [  4.90000000e+01,   1.00000000e+00,   1.22448980e-01],
       [  6.10000000e+01,   1.00000000e+00,   3.27868852e-02],
       [  1.47000000e+02,   1.00000000e+00,   6.12244898e-02],
       [  7.70000000e+01,   1.00000000e+00,   2.59740260e-02],
       [  1.60000000e+02,   1.00000000e+00,   3.75000000e-02],
       [  1.57000000e+02,   7.00000000e+00,   4.45859873e-02],
       [  1.54000000e+02,   4.00000000e+00,   1.29870130e-02]])

In [40]:
f = FunctionFeaturizer(len,
                       longest_run_of_capital_letters,
                       percentage_of_punctuation)

pipe = make_pipeline(f, DecisionTreeClassifier())
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.94113424264178036

In [41]:
from sklearn.metrics import classification_report, confusion_matrix

In [43]:
print(classification_report(pipe.predict(X_test), y_test))

             precision    recall  f1-score   support

        ham       0.97      0.96      0.97      1252
       spam       0.69      0.77      0.73       141

avg / total       0.94      0.94      0.94      1393



_Precision_ is the percentage of instances of the class that are correctly predicted, while _recall_ is the percentage of predictions for that class that are correct.

The _f1-score_ is the _harmonic mean_ of precision and recall.

In [50]:
from textblob import TextBlob

from collections import Counter


class BagOfWordsFeaturizer(TransformerMixin):
    def __init__(self, num_words=None):
        self.num_words = num_words
        
    def fit(self, X, y=None):
        words = []
        for x in X:
            x = TextBlob(x.lower())
            words += [word.lemmatize() for word in x.words]
        if self.num_words:
            words = Counter(words)
            self._vocab = [word for word, _ in words.most_common(self.num_words)]
        else:
            self._vocab = list(set(words))
        return self
    
    def transform(self, X):
        vectors = []
        for x in X:
            x = TextBlob(x.lower())
            word_count = Counter(x.words)
            vector = [0] * len(self._vocab)
            for word, count in word_count.items():
                try:
                    idx = self._vocab.index(word)
                    vector[idx] = count
                except ValueError:
                    pass
            vectors.append(vector)
        return vectors


### Combining multiple featurizers


In [51]:
sms_featurizer = make_union(
    BagOfWordsFeaturizer(20),
    FunctionFeaturizer(len,
                       longest_run_of_capital_letters,
                       percentage_of_punctuation)
)

In [52]:
# to set up textblob:
# pip install textblob
# python -m textblob.download_corpora


In [53]:
pipe = make_pipeline(sms_featurizer, DecisionTreeClassifier())
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9511844938980617